importations

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass      import OneVsOneClassifier
from sklearn.utils import resample

data preparation

In [24]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('./data/urban_development_dataset.csv')
resampled_data = []
classes = df['development_trend_score'].unique()
target_samples = 2000
for cls in classes:
    class_data = df[df['development_trend_score'] == cls]
    if len(class_data) > target_samples:
        resampled = resample(class_data, replace=False, n_samples=target_samples, random_state=42)
    else:
        resampled = resample(class_data, replace=True, n_samples=target_samples, random_state=42)
    resampled_data.append(resampled)

balanced_df = pd.concat(resampled_data)
df = balanced_df
print("Répartition initiale des classes :")
print(df['development_trend_score'].value_counts())

test_data = pd.read_csv('./data/urban_development_test_data.csv')
common_columns = df.columns.intersection(test_data.columns)
#definition of the X and Y
Y = df['development_trend_score']
Y -= 1
X = df[common_columns]

test_data = test_data[common_columns]
le = LabelEncoder()
df.head()


Répartition initiale des classes :
development_trend_score
4    2000
5    2000
2    2000
1    2000
3    2000
Name: count, dtype: int64


,population_density,median_age,average_household_income,number_of_schools,number_of_hospitals,number_of_parks,public_transport_availability,unemployment_rate,retail_density,office_space_availability,...,air_quality_index,crime_rate_per_1000,fire_station_proximity,proximity_to_highways,internet_speed,housing_price_index,year,road_density,urban_area_type,development_trend_score
5846,10854.540185,28.768197,85199.479760,3.0,1.0,3.0,0.968911,6.358546,42.998103,47808.665014,...,52.830152,19.879518,0.669529,18.877326,91.490016,88.356512,2022.0,108.759141,Residential,3
7544,7116.340188,19.603743,37137.715368,7.0,3.0,6.0,0.742236,4.228577,30.537517,66341.178029,...,86.298477,65.531645,6.284401,11.449390,31.791613,107.527199,2022.0,70.697805,Industrial,3
4545,6168.628333,19.555451,63351.767633,9.0,0.0,0.0,NaN,4.145707,45.437759,31054.949220,...,56.146415,31.978132,3.830554,9.470108,61.333742,98.681242,2018.0,64.243977,Mixed,3
6680,4369.467332,38.933783,54554.097575,7.0,3.0,5.0,0.642622,5.976038,47.733647,27306.192868,...,31.514748,18.892706,7.960523,3.198202,35.004245,106.552954,2018.0,NaN,Mixed,3
5855,1403.570521,38.234524,51493.071701,3.0,3.0,1.0,0.171976,4.528690,46.886549,16408.720240,...,39.365717,28.556242,5.401078,17.167561,73.958336,87.267601,2021.0,16.100594,Industrial,3


Preprocessing of Null data


In [25]:
from sklearn.impute import KNNImputer
numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]
# imputer = SimpleImputer(strategy='median') 
# numerical_features = ["population_density",'median_age', 'average_household_income', 'number_of_schools', 'number_of_hospitals','number_of_parks','public_transport_availability' ,'unemployment_rate', 'retail_density','office_space_availability','green_space_percentage','average_temperature','air_quality_index','crime_rate_per_1000','fire_station_proximity','proximity_to_highways','internet_speed','road_density','housing_price_index','year',  ]
# df[numerical_features] = imputer.fit_transform(df[numerical_features])
# df.isnull().sum()

knn_imputer = KNNImputer(n_neighbors=5)  
X[numerical_features] = knn_imputer.fit_transform(X[numerical_features])
test_data[numerical_features] = knn_imputer.fit_transform(test_data[numerical_features])
print(X.isnull().sum())


/tmp/ipykernel_108947/3869585995.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = knn_imputer.fit_transform(X[numerical_features])


population_density               0
median_age                       0
average_household_income         0
number_of_schools                0
number_of_hospitals              0
number_of_parks                  0
public_transport_availability    0
unemployment_rate                0
retail_density                   0
office_space_availability        0
green_space_percentage           0
average_temperature              0
air_quality_index                0
crime_rate_per_1000              0
fire_station_proximity           0
proximity_to_highways            0
internet_speed                   0
housing_price_index              0
year                             0
road_density                     0
dtype: int64


normalization of data

In [26]:
from tensorflow.keras.utils import to_categorical
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
# Y = le.fit_transform(Y)
# Y = to_categorical(Y)
scaled_data = scaler.transform(test_data)
test_data_scaled = pd.DataFrame(scaled_data, columns=test_data.columns, index=test_data.index)



check outliers

In [27]:
from scipy.stats import zscore
z_scores = np.abs(zscore(X_scaled)) 
outlier_mask = (z_scores > 3).any(axis=1)
plt.figure(figsize=(10, 6))
print(z_scores)

      population_density  median_age  average_household_income  \
5846            3.005547    0.709511                  0.467174   
7544            1.098085    1.672421                  0.394031   
4545            0.614504    1.677495                  0.075691   
6680            0.303540    0.358588                  0.081952   
5855            1.816925    0.285117                  0.136801   
...                  ...         ...                       ...   
6520            0.337106    0.030071                  0.194442   
8482            1.970142    0.158172                  0.132746   
6877            0.730339    0.983954                  0.122113   
4907            1.158970    0.778371                  0.216804   
9991            0.396137    0.547157                  0.580031   

      number_of_schools  number_of_hospitals  number_of_parks  \
5846           0.904290             0.734323         0.042640   
7544           0.921857             0.724780         1.665695   
4545        

<Figure size 1000x600 with 0 Axes>

trying some changes on the data

In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

model = XGBClassifier()
model.fit(X_scaled, Y)
importances = model.feature_importances_
threshold = 0.
important_features = importances > threshold
X_selected = X_scaled[:, important_features]
test_data_selected = test_data.iloc[:, important_features]

print(f"Selected Features: {len(important_features)}")

# Optional: View the first few rows of the selected data
print("Selected data (X):")
print(X_selected[:5])
print("Selected test data:")
print(test_data_selected.head())

In [52]:
X_scaled['community_services'] = X_scaled['number_of_schools'] + X_scaled['number_of_hospitals'] + X_scaled['number_of_parks']
X_scaled['accessibility'] = X_scaled['public_transport_availability'] * X_scaled['road_density']
X_scaled.shape
X_new = X_scaled
X_new = X_scaled.drop(columns=['number_of_schools', 'number_of_hospitals', 'number_of_parks', 'public_transport_availability', 'road_density'])

test_data_scaled['community_services'] = test_data_scaled['number_of_schools'] + test_data_scaled['number_of_hospitals'] + test_data_scaled['number_of_parks']
test_data_scaled['accessibility'] = test_data_scaled['public_transport_availability'] * test_data_scaled['road_density']
test_data_scaled.shape
test_data_new = test_data_scaled.drop(columns=['number_of_schools', 'number_of_hospitals', 'number_of_parks', 'public_transport_availability', 'road_density'])

split data

In [49]:
train_x, test_x, train_y, test_y = train_test_split(X_new, Y, test_size=0.2, random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
# train_x, train_y = X, Y


(8000, 17) (2000, 17) (8000,) (2000,)


SVM model

In [30]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


model = svm.SVC(kernel='linear', gamma='scale' , C=0.1)
model.fit(train_x,train_y)

# ovo_model = OneVsOneClassifier(model)
# ovo_model.fit(train_x, train_y)
y_pred = model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")

Test Accuracy: 0.215
Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.26      0.30      0.28       415
           1       0.18      0.14      0.16       395
           2       0.19      0.16      0.17       387
           3       0.22      0.25      0.23       410
           4       0.20      0.22      0.21       393

    accuracy                           0.21      2000
   macro avg       0.21      0.21      0.21      2000
weighted avg       0.21      0.21      0.21      2000



rf model 

In [50]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# rfm = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=100, random_state=42)
rfm = RandomForestClassifier(random_state=42, n_estimators=200)
# rfm.fit(X, Y)
rfm.fit(train_x, train_y)
y_pred_rf = rfm.predict(test_x)

print(f"Test Accuracy for Random Forest: {accuracy_score(test_y, y_pred_rf)}")
print(f"Classification Report for Random Forest:\n{classification_report(test_y, y_pred_rf)}")


Test Accuracy for Random Forest: 0.61
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.91      0.79      0.85       415
           1       0.34      0.42      0.38       395
           2       0.34      0.42      0.38       387
           3       0.72      0.57      0.64       410
           4       0.93      0.84      0.88       393

    accuracy                           0.61      2000
   macro avg       0.65      0.61      0.62      2000
weighted avg       0.65      0.61      0.63      2000



NN model 

In [37]:
# print(train_y.shape)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)


model = Sequential([
    Dense(128, activation='relu', input_shape=(train_x.shape[1],)),  
    Dropout(0.3),  
    Dense(64, activation='relu'), 
    Dropout(0.3),
    Dense(train_y.shape[1], activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_x, train_y, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# Évaluation du modèle
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(f"Loss sur le test : {test_loss:.4f}")
print(f"Précision sur le test : {test_accuracy:.4f}")



# y_true = np.argmax(test_y, axis=1)
# print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
# print(f"Classification Report:\n{classification_report(y_true, y_pred)}")
# # Prédictions
# y_pred = model.predict(test_x)
# print(f"Test Accuracy for Random Forest: {accuracy_score(test_y, y_pred_rf)}")
# print(f"Classification Report for Random Forest:\n{classification_report(test_y, y_pred_rf)}")


Epoch 1/50


/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1991 - loss: 1.6975 - val_accuracy: 0.2194 - val_loss: 1.6110
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2188 - loss: 1.6148 - val_accuracy: 0.2113 - val_loss: 1.6080
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2324 - loss: 1.6085 - val_accuracy: 0.2288 - val_loss: 1.6056
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2507 - loss: 1.5907 - val_accuracy: 0.2288 - val_loss: 1.6021
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2615 - loss: 1.5825 - val_accuracy: 0.2544 - val_loss: 1.5992
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2637 - loss: 1.5800 - val_accuracy: 0.2419 - val_loss: 1.5964
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2845 - loss: 1.5639 - val_accuracy: 0.2412 - val_loss: 1.5950
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2880 - loss: 1.5604 - val_accuracy: 0.2444 - val_

In [43]:
y_pred = np.argmax(model.predict(test_x), axis=1)
y_pred +=1
y_true = np.argmax(test_y, axis=1)
y_true +=1
y_true[:10], y_pred[:10]
print(f"Test Accuracy for NN: {accuracy_score(y_true, y_pred)}")
print(f"Classification Report for Random Forest:\n{classification_report(y_true, y_pred)}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Test Accuracy for NN: 0.383
Classification Report for Random Forest:
              precision    recall  f1-score   support

           1       0.51      0.51      0.51       415
           2       0.26      0.18      0.21       395
           3       0.26      0.26      0.26       387
           4       0.44      0.34      0.38       410
           5       0.41      0.62      0.49       393

    accuracy                           0.38      2000
   macro avg       0.37      0.38      0.37      2000
weighted avg       0.38      0.38      0.37      2000



export result to csv from the test data

In [53]:
# output = np.argmax(model.predict(test_data), axis=1)
# output += 1
output = rfm.predict(test_data_new) #here changing the model 
output += 1
submission = pd.DataFrame({
    "ID": test_data.index + 1,  
    "development_trend_score": output
})
submission_file_path = './test/development_trend_predictions_classification.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved at: {submission_file_path}")

Submission file saved at: ./test/development_trend_predictions_classification.csv


In [16]:
# Train a Gaussian Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(train_x, train_y)

# Predict on the test set
nb_y_pred = nb_model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, nb_y_pred)}")
print(f"Classification Report for Gaussian Naive Bayes:\n{classification_report(test_y, nb_y_pred)}")

Test Accuracy: 0.3995
Classification Report for Gaussian Naive Bayes:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       184
           1       0.12      0.01      0.01       387
           2       0.40      0.99      0.57       808
           3       0.00      0.00      0.00       410
           4       0.00      0.00      0.00       211

    accuracy                           0.40      2000
   macro avg       0.11      0.20      0.12      2000
weighted avg       0.19      0.40      0.23      2000



/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",